In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import keras.backend as K
from keras.layers import *
from keras.models import *
from keras.utils import *
from keras.regularizers import *
import numpy as np
import _pickle as pickle
import uuid

Using TensorFlow backend.


In [2]:
embed = hub.Module('./embeddings/GUSE')

INFO:tensorflow:Using /var/folders/mq/k5hvs9cj1vb8q7x9830z516r0000gn/T/tfhub_modules to cache modules.
INFO:tensorflow:Initialize variable module/Embeddings_en/sharded_0:0 from checkpoint b'./embeddings/GUSE/variables/variables' with Embeddings_en/sharded_0
INFO:tensorflow:Initialize variable module/Embeddings_en/sharded_1:0 from checkpoint b'./embeddings/GUSE/variables/variables' with Embeddings_en/sharded_1
INFO:tensorflow:Initialize variable module/Embeddings_en/sharded_10:0 from checkpoint b'./embeddings/GUSE/variables/variables' with Embeddings_en/sharded_10
INFO:tensorflow:Initialize variable module/Embeddings_en/sharded_11:0 from checkpoint b'./embeddings/GUSE/variables/variables' with Embeddings_en/sharded_11
INFO:tensorflow:Initialize variable module/Embeddings_en/sharded_12:0 from checkpoint b'./embeddings/GUSE/variables/variables' with Embeddings_en/sharded_12
INFO:tensorflow:Initialize variable module/Embeddings_en/sharded_13:0 from checkpoint b'./embeddings/GUSE/variables/

INFO:tensorflow:Initialize variable module/Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_1/self_attention/layer_prepostprocess/layer_norm/layer_norm_scale:0 from checkpoint b'./embeddings/GUSE/variables/variables' with Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_1/self_attention/layer_prepostprocess/layer_norm/layer_norm_scale
INFO:tensorflow:Initialize variable module/Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_1/self_attention/multihead_attention/k/kernel:0 from checkpoint b'./embeddings/GUSE/variables/variables' with Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_1/self_attention/multihead_attention/k/kernel
INFO:tensorflow:Initialize variable module/Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_1/self_attention/multihead_attention/output_transform/kernel:0 from checkpoint b'./embeddings/GUSE/variables/variables' with Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_1/self_attention/multihead_attention/out

INFO:tensorflow:Initialize variable module/Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_3/self_attention/multihead_attention/q/kernel:0 from checkpoint b'./embeddings/GUSE/variables/variables' with Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_3/self_attention/multihead_attention/q/kernel
INFO:tensorflow:Initialize variable module/Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_3/self_attention/multihead_attention/v/kernel:0 from checkpoint b'./embeddings/GUSE/variables/variables' with Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_3/self_attention/multihead_attention/v/kernel
INFO:tensorflow:Initialize variable module/Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_4/ffn/conv1/bias:0 from checkpoint b'./embeddings/GUSE/variables/variables' with Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_4/ffn/conv1/bias
INFO:tensorflow:Initialize variable module/Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_4/ffn

INFO:tensorflow:Initialize variable module/Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_prepostprocess/layer_norm/layer_norm_scale:0 from checkpoint b'./embeddings/GUSE/variables/variables' with Encoder_en/Transformer/TransformerEncodeFast/encoder/layer_prepostprocess/layer_norm/layer_norm_scale
INFO:tensorflow:Initialize variable module/Encoder_en/Transformer/dense/bias:0 from checkpoint b'./embeddings/GUSE/variables/variables' with Encoder_en/Transformer/dense/bias
INFO:tensorflow:Initialize variable module/Encoder_en/Transformer/dense/kernel:0 from checkpoint b'./embeddings/GUSE/variables/variables' with Encoder_en/Transformer/dense/kernel
INFO:tensorflow:Initialize variable module/Encoder_en/hidden_layers/tanh_layer_0/dense/bias:0 from checkpoint b'./embeddings/GUSE/variables/variables' with Encoder_en/hidden_layers/tanh_layer_0/dense/bias
INFO:tensorflow:Initialize variable module/Encoder_en/hidden_layers/tanh_layer_0/dense/kernel:0 from checkpoint b'./embeddings/GUS

In [3]:
def GUSE(param):
    return embed(tf.squeeze(tf.cast(param, tf.string)), signature="default", as_dict=True)["default"]

In [4]:
input_layer = Input(shape=(1,), dtype="string")
guse = Lambda(GUSE, output_shape=(512,))(input_layer)
dense1 = Dense(256, activation="tanh", kernel_regularizer=l2(0.01))(guse)
dropout1 = Dropout(0.3)(dense1)
dense2 = Dense(128, activation="tanh", kernel_regularizer=l2(0.01))(dropout1)
dropout2 = Dropout(0.3)(dense2)
dense3 = Dense(128, activation="tanh", kernel_regularizer=l2(0.01))(dropout2)
dropout3 = Dropout(0.3)(dense3)
dense4 = Dense(64, activation="tanh", kernel_regularizer=l2(0.01))(dropout3)
dropout4 = Dropout(0.3)(dense4)
output = Dense(1, activation="sigmoid")(dropout4)
model = Model(inputs=[input_layer], outputs=output)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1)                 0         
_________________________________________________________________
lambda_1 (Lambda)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               16512     
__________

In [5]:
session = tf.Session()
K.set_session(session)
session.run(tf.global_variables_initializer())
session.run(tf.tables_initializer())
model.load_weights("guse_model.h5")

In [6]:
def predict_chance(sentence, model):
    filler_arr = [sentence, ""]
    np_filler = np.asarray(filler_arr)
    np_filler = np.reshape(np_filler, (np_filler.shape[0],1))
    pred = model.predict(np_filler)
    chance = float(pred[0])*100
    return chance

In [7]:
predict_chance("I hope to deliver this project on time", model)

35.216304659843445

In [8]:
predict_chance("Nobody truly knows the theory of everything", model)

18.275001645088196

In [9]:
predict_chance("I felt lonely even though I was surrounded by many people", model)

96.4241087436676

In [10]:
predict_chance("Common sense is likely the rarest virtue", model)

93.11326742172241

In [11]:
predict_chance("The meeting between procrastinators started early", model)

66.78162217140198